In [1]:
import pandas as pd
data = pd.read_csv('data/train_set/2.4pastAvg_diff.csv', header = 0)
data.columns

Index(['broadcast_date', 'broadcast_time', 'broadcast_code', 'product_code',
       'product_name', 'product_category', 'unit_price', 'handling_amount',
       'off_day', 'date', 'time', 'weekday', 'hour', 'temperature_seoul',
       'rain_seoul', 'humid_seoul', 'snow_seoul', 'temperature_busan',
       'rain_busan', 'humid_busan', 'snow_busan', 'pmWarning_seoul',
       'pmWarning_busan', 'season', 'day_dup', 'week_dup', 'max_watching',
       'avg_watching', 'working_time', 'pastAvg_diff'],
      dtype='object')

In [2]:
## 1. 사용할 데이터만 가져오기

In [11]:
data_use = data[['broadcast_time','product_category', 'unit_price', 
       'off_day',  'weekday', 'hour', 'temperature_seoul',
       'rain_seoul', 'humid_seoul', 'snow_seoul', 'temperature_busan',
       'rain_busan', 'humid_busan', 'snow_busan', 'pmWarning_seoul',
       'pmWarning_busan', 'season', 'day_dup', 'week_dup', 'max_watching',
       'avg_watching', 'working_time', 'pastAvg_diff','handling_amount']]
data_use_fit = data_use

## 2. 데이터의 종류
       
#### 정규화를 해야할 것 : 'broadcast_time', 'unit_price','rain_seoul', 'humid_seoul', 'snow_seoul', 'temperature_busan', 'rain_busan', 'humid_busan', 'snow_busan','hour','temperature_seoul','day_dup', 'week_dup','max_watching','avg_watching', 

#### 1 of c : 'product_category', 'weekday','season',

#### 그대로 사용 : 'off_day','pmWarning_seoul', 'pmWarning_busan', 'working_time', 

#### 정답 :  'pastAvg_diff','handling_amount'

#### standard_scalar_list

In [12]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

standard_scalar_list = [ 'broadcast_time', 'unit_price','rain_seoul',
                        'humid_seoul', 'snow_seoul', 'temperature_busan',
                        'rain_busan', 'humid_busan', 'snow_busan','hour',
                        'temperature_seoul','day_dup', 'week_dup','max_watching','avg_watching',]

data_use_fit[standard_scalar_list] = std_scaler.fit_transform(pd.DataFrame(data_use[standard_scalar_list]))
data_use_fit

C:\Users\dlwlg\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\dlwlg\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,broadcast_time,product_category,unit_price,off_day,weekday,hour,temperature_seoul,rain_seoul,humid_seoul,snow_seoul,...,pmWarning_seoul,pmWarning_busan,season,day_dup,week_dup,max_watching,avg_watching,working_time,pastAvg_diff,handling_amount
0,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-1.012871,-1.271887,-0.791641,-0.564382,False,NaN,2099000.0
1,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-1.012871,-1.271887,-0.791641,-0.564382,False,NaN,4371000.0
2,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-0.197372,-1.032358,-0.791641,-0.564382,False,1.163000e+06,3262000.0
3,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-0.197372,-1.032358,-0.791641,-0.564382,False,2.584000e+06,6955000.0
4,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,0.618128,-0.792829,-0.791641,-0.564382,False,3.991500e+06,6672000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,-0.123021,주방,-0.437520,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-1.012871,-0.553299,-0.791641,-0.564382,False,-1.078606e+07,1664000.0
37364,-0.123021,주방,-0.437520,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-0.197372,-0.313770,0.845573,2.000830,False,-1.956000e+06,10157000.0
37365,-0.123021,주방,-0.423849,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-0.197372,-0.313770,0.845573,2.000830,False,-3.341438e+06,2328000.0
37366,-0.123021,주방,-0.396507,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-0.197372,-0.313770,0.845573,2.000830,False,-6.707062e+06,9149000.0


## 더미코딩

In [13]:
data_use_fit['product_category'] = list(map(str,data_use_fit['product_category']))
data_use_fit['weekday'] = list(map(str,data_use_fit['weekday']))
data_use_fit['season'] = list(map(str,data_use_fit['season']))

data_use_fit


C:\Users\dlwlg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\dlwlg\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dlwlg\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,broadcast_time,product_category,unit_price,off_day,weekday,hour,temperature_seoul,rain_seoul,humid_seoul,snow_seoul,...,pmWarning_seoul,pmWarning_busan,season,day_dup,week_dup,max_watching,avg_watching,working_time,pastAvg_diff,handling_amount
0,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-1.012871,-1.271887,-0.791641,-0.564382,False,NaN,2099000.0
1,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-1.012871,-1.271887,-0.791641,-0.564382,False,NaN,4371000.0
2,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-0.197372,-1.032358,-0.791641,-0.564382,False,1.163000e+06,3262000.0
3,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,-0.197372,-1.032358,-0.791641,-0.564382,False,2.584000e+06,6955000.0
4,-0.123021,의류,-0.585305,True,1,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,...,False,False,1,0.618128,-0.792829,-0.791641,-0.564382,False,3.991500e+06,6672000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,-0.123021,주방,-0.437520,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-1.012871,-0.553299,-0.791641,-0.564382,False,-1.078606e+07,1664000.0
37364,-0.123021,주방,-0.437520,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-0.197372,-0.313770,0.845573,2.000830,False,-1.956000e+06,10157000.0
37365,-0.123021,주방,-0.423849,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-0.197372,-0.313770,0.845573,2.000830,False,-3.341438e+06,2328000.0
37366,-0.123021,주방,-0.396507,False,1,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,...,False,False,1,-0.197372,-0.313770,0.845573,2.000830,False,-6.707062e+06,9149000.0


In [14]:
data_use_fit = pd.get_dummies(data_use_fit, drop_first=True, columns = ['product_category', 'weekday','season'])
data_use_fit.columns

Index(['broadcast_time', 'unit_price', 'off_day', 'hour', 'temperature_seoul',
       'rain_seoul', 'humid_seoul', 'snow_seoul', 'temperature_busan',
       'rain_busan', 'humid_busan', 'snow_busan', 'pmWarning_seoul',
       'pmWarning_busan', 'day_dup', 'week_dup', 'max_watching',
       'avg_watching', 'working_time', 'pastAvg_diff', 'handling_amount',
       'product_category_가전', 'product_category_건강기능', 'product_category_농수축',
       'product_category_생활용품', 'product_category_속옷', 'product_category_의류',
       'product_category_이미용', 'product_category_잡화', 'product_category_주방',
       'product_category_침구', 'weekday_1', 'weekday_2', 'weekday_3',
       'weekday_4', 'weekday_5', 'weekday_6', 'season_2', 'season_3',
       'season_4'],
      dtype='object')

In [15]:
data_use_fit

,broadcast_time,unit_price,off_day,hour,temperature_seoul,rain_seoul,humid_seoul,snow_seoul,temperature_busan,rain_busan,...,product_category_침구,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,season_2,season_3,season_4
0,-0.123021,-0.585305,True,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,-2.307609,-0.152131,...,0,1,0,0,0,0,0,0,0,0
1,-0.123021,-0.585305,True,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,-2.307609,-0.152131,...,0,1,0,0,0,0,0,0,0,0
2,-0.123021,-0.585305,True,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,-2.307609,-0.152131,...,0,1,0,0,0,0,0,0,0,0
3,-0.123021,-0.585305,True,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,-2.307609,-0.152131,...,0,1,0,0,0,0,0,0,0,0
4,-0.123021,-0.585305,True,-1.255439,-2.124309,-0.126626,0.240680,-0.064213,-2.307609,-0.152131,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,-0.123021,-0.437520,False,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,-2.384841,-0.152131,...,0,1,0,0,0,0,0,0,0,0
37364,-0.123021,-0.437520,False,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,-2.384841,-0.152131,...,0,1,0,0,0,0,0,0,0,0
37365,-0.123021,-0.423849,False,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,-2.384841,-0.152131,...,0,1,0,0,0,0,0,0,0,0
37366,-0.123021,-0.396507,False,1.312840,-2.076522,-0.126626,-0.504288,-0.064213,-2.384841,-0.152131,...,0,1,0,0,0,0,0,0,0,0


In [16]:
data_use_fit.to_csv('data/train_set/3.data_normalize.csv',index = False)

0           테이트 남성 셀린니트3종
1           테이트 남성 셀린니트3종
2           테이트 남성 셀린니트3종
3        오모떼 레이스 파운데이션 브라
4        오모떼 레이스 파운데이션 브라
               ...       
37367       무이자쿠첸압력밥솥 6인용
37368       일시불쿠첸압력밥솥 6인용
37369      일시불쿠첸압력밥솥 10인용
37370       무이자쿠첸압력밥솥 6인용
37371       일시불쿠첸압력밥솥 6인용
Name: 상품명, Length: 37372, dtype: object